# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [39]:
def find_full_team_name(name, name_s):
    if (isinstance(name, str)):
        match = name_s[name_s.str.contains(name)]
        if len(match) > 0: return match.iloc[0]
    return name

def sport_population_per_region(sport):
    filename = "assets/{}.csv".format(sport)

    sport_df = pd.read_csv(filename)

    with open('assets/wikipedia_data.html', 'r', encoding='utf-8') as file:
        cities_html_content = file.read()

    # import cities_df
    # replace <br> and <br/> with a unique delimiter
    cities_html_content = cities_html_content.replace('<br>', ';').replace('<br/>', ';')
    cities=pd.read_html(cities_html_content)[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    clean_pattern = "\[.*\]|\(.*\)|[*+]"

    # clean cities_df
    cities = cities.rename(columns={"Population ;(2016 est.)[8]": "Population"})
    cities = cities[["Metropolitan area", "Population", sport]]
    cities[["Metropolitan area", sport]] = cities[["Metropolitan area", sport]].replace(clean_pattern, "", regex=True)
    cities[["Metropolitan area", sport]] = cities[["Metropolitan area", sport]].apply(lambda col: col.str.strip())
    cities[sport].replace("^\s*$|^—*$", np.nan, inplace=True, regex=True)
    cities["Population"] = pd.to_numeric(cities["Population"])
    cities = cities.dropna()

    # clean sport_df and filter for 2018
    sport_df["team"].replace(clean_pattern, "", regex=True, inplace=True)
    sport_df["team"] = sport_df["team"].str.strip()
    sport_df = sport_df[sport_df["year"] == 2018]
    sport_df = sport_df[["team", "W", "L"]]
    remove_rows = sport_df["team"].isin(["Atlantic Division", "Metropolitan Division",
                                         "Pacific Division", "Central Division",
                                         "Northwest Division", 
                                         "Southeast Division", "Southwest Division",
                                         "AFC East", "AFC North", "AFC South", "AFC West",
                                         "NFC East", "NFC North", "NFC South", "NFC West",
                                         ])
    sport_df = sport_df.drop(index=sport_df[remove_rows].index)
    sport_df["W"] = pd.to_numeric(sport_df["W"])
    sport_df["L"] = pd.to_numeric(sport_df["L"])

    # separate teams from the same region in HTML file
    cities_sport_df = cities[["Metropolitan area", sport]].copy()
    cities_sport_df[sport] = cities_sport_df[sport].str.split(";")
    cities_sport_df = cities_sport_df.explode(sport)

    # left-merge cities_df with sport_df 
    cities_sport_df["Full Team Name"] = cities_sport_df[sport].apply(lambda name: find_full_team_name(name, sport_df["team"]))
    cities_sport_df = cities_sport_df.merge(sport_df, how="left", left_on="Full Team Name", right_on="team").loc[:, ["Metropolitan area", sport, "W", "L"]]

    # calculate W/L Ratio and average per region
    cities_sport_df["Ratio W/L"] = cities_sport_df["W"]/(cities_sport_df["W"] + cities_sport_df["L"])
    cities_wlratio = cities_sport_df.groupby("Metropolitan area").agg({"Ratio W/L": np.mean}).sort_values(by="Metropolitan area")

    population_by_region_df = cities.sort_values(by="Metropolitan area")
    win_loss_by_region_df = cities_wlratio.sort_values(by="Metropolitan area")
    
    return [population_by_region_df, win_loss_by_region_df]
    



In [30]:
def nhl_correlation(): 
    [population_by_region_df, win_loss_by_region_df] = sport_population_per_region("NHL")

    population_by_region = population_by_region_df["Population"]
    win_loss_by_region = win_loss_by_region_df["Ratio W/L"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

In [31]:
nhl_correlation()

PearsonRResult(statistic=0.012486162921209881, pvalue=0.9497182859911808)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [33]:
def nba_correlation():    
    [population_by_region_df, win_loss_by_region_df] = sport_population_per_region("NBA")

    population_by_region = population_by_region_df["Population"]
    win_loss_by_region = win_loss_by_region_df["Ratio W/L"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [32]:
nba_correlation()

PearsonRResult(statistic=-0.17657160252844611, pvalue=0.36874741604462974)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [34]:
def mlb_correlation(): 
    [population_by_region_df, win_loss_by_region_df] = sport_population_per_region("MLB")

    population_by_region = population_by_region_df["Population"]
    win_loss_by_region = win_loss_by_region_df["Ratio W/L"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [35]:
mlb_correlation()

PearsonRResult(statistic=0.1502769830266931, pvalue=0.46370703378875616)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [36]:
def nfl_correlation(): 
    [population_by_region_df, win_loss_by_region_df] = sport_population_per_region("NFL")

    population_by_region = population_by_region_df["Population"]
    win_loss_by_region = win_loss_by_region_df["Ratio W/L"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [37]:
nfl_correlation()

PearsonRResult(statistic=0.004922112149349456, pvalue=0.9797833458363692)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [58]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    [_, win_loss_by_region_nfl_df] = sport_population_per_region("NFL")
    [_, win_loss_by_region_mlb_df] = sport_population_per_region("MLB")
    [_, win_loss_by_region_nhl_df] = sport_population_per_region("NHL")
    [_, win_loss_by_region_nba_df] = sport_population_per_region("NBA")

    win_loss_by_region_nfl_df = win_loss_by_region_nfl_df.rename(columns={"Ratio W/L": "NFL"})
    win_loss_by_region_mlb_df = win_loss_by_region_mlb_df.rename(columns={"Ratio W/L": "MLB"})
    win_loss_by_region_nhl_df = win_loss_by_region_nhl_df.rename(columns={"Ratio W/L": "NHL"})
    win_loss_by_region_nba_df = win_loss_by_region_nba_df.rename(columns={"Ratio W/L": "NBA"})

    wl_ratio_per_region_per_sport = pd.concat([win_loss_by_region_nfl_df, win_loss_by_region_mlb_df, win_loss_by_region_nhl_df, win_loss_by_region_nba_df], axis=1)

    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    for i in range(len(sports)):
        for j in range(i+1, len(sports)):
            column_name1 = sports[i]
            column_name2 = sports[j]

            data = wl_ratio_per_region_per_sport[[column_name1, column_name2]]
            data = data.dropna()
            
            p_values.loc[column_name1, column_name2] = stats.ttest_rel(data[column_name1], data[column_name2]).pvalue
            p_values.loc[column_name2, column_name1] = p_values.loc[column_name1, column_name2]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [59]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
